# Setting up

In [9]:
import pandas as pd
import re
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.stem import WordNetLemmatizer
import pymorphy2
import warnings
import nltk
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [10]:
# Preprocessing data function
# Arguments:
#   - pandas Series we want to preprocess and list of stop words
#        data : pd.Series, stop_words : list
# Rerutns:
#   - preprocesse pandas series
#       (pd.Series)
def preprocess_data(data : pd.Series, stop_words : list):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\)\w+', ' ', x))
    data = data.apply(lambda x: re.sub(r'\b\d+\b', ' ', x))
    data = data.apply(lambda x: re.sub(r'\w+\(', ' ', x))
    data = data.apply(lambda x: re.sub(r'[-–\.\*\?«»•\(\)/—,:;\"]', ' ', x))
    data = data.apply(lambda x: re.sub(r'\d+\.\d+\.', ' ', x))
    data = data.apply(lambda x: re.sub(r'\d+\.\d+', ' ', x))
    data = data.apply(lambda x: re.sub(r'\d+\.', ' ', x))
    data = data.apply(lambda x: re.sub(r'\s+', ' ', x))
    data = data.apply(lambda x : ' '.join([word for word in x.split() if word not in stop_words]))
  
    return data

In [11]:
data = pd.read_csv('Data/vacancies_data.csv', index_col=0)

In [12]:
itmo_data = pd.read_csv('Data/Itmo_with_keywords_30.04.2020.csv', index_col=0)

In [13]:
# getting stop words
stop_words = nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english') +\
['основа', 'введение', 'базовый', 'современный', 'начальный', 'зарплата']

# Preparing and tokenizing data

In [14]:
# cleaning hh data set
data['Ключевые навыки clean'] = preprocess_data(data['Ключевые навыки'], stop_words)
itmo_data['Keywords clean'] = preprocess_data(itmo_data['Keywords'], stop_words)

# getting only vacancies with key words
data = data[data['Ключевые навыки clean'] != ' ']

In [15]:
en_lemmatizer = WordNetLemmatizer()
ru_lemmatizer = pymorphy2.analyzer.MorphAnalyzer()

In [16]:
# tokenizing both data sets
data['tokenized'] = data['Ключевые навыки clean'].apply(lambda x : 
                                                  list(set([ru_lemmatizer.parse(word)[0].normal_form if 
                                                            re.search(r'[А-Яа-я]+', word) else
                                                            en_lemmatizer.lemmatize(word) for word in x.split()])))

In [17]:
itmo_data['tokenized'] = itmo_data['Keywords clean'].apply(lambda x : 
                                                  list(set([ru_lemmatizer.parse(word)[0].normal_form if 
                                                            re.search(r'[А-Яа-я]+', word) else
                                                            en_lemmatizer.lemmatize(word) for word in x.split()])))

In [18]:
itmo_data['New_subj'] = itmo_data['SUBJECT'].apply(lambda x: [ru_lemmatizer.parse(word)[0].normal_form if 
                                                            re.search(r'[А-Яа-я]+', word) else
                                                            en_lemmatizer.lemmatize(word) for word in x.split()])

In [19]:
itmo_data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,Keywords,Keywords clean,tokenized,New_subj
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,Элементы линейной алгебры; Математический анал...,элементы линейной алгебры математический анали...,"[вероятность, элемент, анализ, переменный, оди...",[математика]
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,Элементы линейной алгебры; Математический анал...,элементы линейной алгебры математический анали...,"[вероятность, элемент, анализ, переменный, оди...",[математика]
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,Механика; Механические колебания и волны; Элем...,механика механические колебания волны элементы...,"[электромагнитный, элемент, электростатика, оп...",[физик]
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,Механика; Механические колебания и волны; Элем...,механика механические колебания волны элементы...,"[электромагнитный, элемент, электростатика, оп...",[физик]
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,,,[],"[основа, право]"


# Performing doc2vec matching

In [20]:
import numpy as np

In [21]:
# tagging documents from hh and itmo data sets
documents_hh = [TaggedDocument(doc, [' '.join(doc)]) for doc in data['tokenized'] if doc != []]
documents_itmo = [TaggedDocument(doc, [' '.join(doc)]) for doc in np.unique(itmo_data['tokenized'])]

In [22]:
# constructing generic dataset
documents = documents_hh + documents_itmo

In [23]:
model = Doc2Vec(documents, dm=0, alpha=0.025, size=20, min_alpha=0.025, min_count=0)

In [24]:
for epoch in tqdm(range(10)):
    model.train(documents, total_examples=model.corpus_count, epochs=10)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


In [68]:
vac = []

for vacancy in tqdm(data.values):
    most_sim = 0
    vacn = ' '
    
    desc_sim = []
    for desc in itmo_data.values:
        sim = model.docvecs.similarity(' '.join(desc[-2]), ' '.join(vacancy[-1]))
        desc_sim.append((desc, sim))
    
    new_desc_sim = []
    for item in sorted(desc_sim, key=lambda x:x[1], reverse=True)[:10]:
        curr_weight = 0
        for word in item[0][-2]:
            if word == 'с++':
                if re.findall(r'c\+\+', ' '.join(vacancy[-1])):
                    curr_weight += len(re.findall(r'c\+\+', ' '.join(vacancy[-1])))
            elif word != ' ' and word != '' and word != '+':
                if re.findall(word, ' '.join(vacancy[-1])):
                    curr_weight += len(re.findall(word, ' '.join(vacancy[-1])))
                
        if len(vacancy[-1]) != 0 or len(item[0][-2]) != 0:
            curr_weight /= ((len(vacancy[-1]) + len(item[0][-2])))
        else:
            curr_weight = 0
        
        
        new_desc_sim.append((item[0], vacancy, curr_weight))
    
    predicted = sorted(new_desc_sim, key=lambda x:x[-1], reverse=True)[0]
    vac.append(predicted if predicted[-1] > 0.1 else ' ')
    

100%|██████████| 14369/14369 [2:21:46<00:00,  1.69it/s] 


In [69]:
vac = np.array(vac)

In [95]:
vac = vac[vac != ' ']

In [100]:
SUBFIELDCODE = []
SUBFIELDNAME = []
DEGREE = []
SUBJECT = []
SEMESTER = []
START_YEAR = []
KEYWORDS = []
JOBNAME = []
ORGANIZATION = []
LOCATION = []
WORKTIME = []
EXPERIANCE = []
SALARY = []
REQUERMENTS = []

for record in tqdm(vac):
    SUBFIELDCODE.append(record[0][0])
    SUBFIELDNAME.append(record[0][1])
    DEGREE.append(record[0][2])
    SUBJECT.append(record[0][3])
    SEMESTER.append(record[0][4])
    START_YEAR.append(record[0][5])
    KEYWORDS.append(record[0][6])
    
    JOBNAME.append(record[1][0])
    ORGANIZATION.append(record[1][1])
    LOCATION.append(record[1][2])
    WORKTIME.append(record[1][3])
    EXPERIANCE.append(record[1][4])
    SALARY.append(record[1][5])
    REQUERMENTS.append(record[1][6])

100%|██████████| 1019/1019 [00:00<00:00, 277478.14it/s]


In [112]:
matched = pd.DataFrame({'SUBFIELDCODE' : SUBFIELDCODE, 'SUBFIELDNAME' : SUBFIELDNAME, 'DEGREE': DEGREE, 
                      'SUBJECT' : SUBJECT, 'SEMESTER' : SEMESTER, 'START_YEAR' : START_YEAR, 'KEYWORDS' : KEYWORDS,
                       'JOBNAME' : JOBNAME, 'ORGANIZATION' : ORGANIZATION, 'LOCATION' : LOCATION, 
                       'WORKTIME' : WORKTIME, 'EXPERIANCE' : EXPERIANCE, 'SALARY': SALARY, 
                       'REQUERMENTS' : REQUERMENTS})

In [103]:
matched.iloc[0]

SUBFIELDCODE                                             27.04.05
SUBFIELDNAME                                 Научная коммуникация
DEGREE                                                    Магистр
SUBJECT              Основы научной визуализации и анализа данных
SEMESTER                                                        3
START_YEAR                                                   2018
KEYWORDS        Простейшие функции MS Excel; Введение в анализ...
JOBNAME              Оператор видеонаблюдения (кассовый контроль)
ORGANIZATION                                              Буквоед
LOCATION        Площадь Ленина, Санкт-Петербург, Минеральная у...
WORKTIME                            Полная занятость, полный день
EXPERIANCE                                           не требуется
SALARY                           от 35 000 до 40 000 руб. на руки
REQUERMENTS                      Работа с базами данных, MS Excel
Name: 0, dtype: object

In [104]:
matched.iloc[1]

SUBFIELDCODE                                             27.04.07
SUBFIELDNAME                     Промышленный дизайн и инжиниринг
DEGREE                                                    Магистр
SUBJECT                           Современный промышленный дизайн
SEMESTER                                                        2
START_YEAR                                                   2015
KEYWORDS        Adobe Photoshop; Коллаж (базовый уровень); Осн...
JOBNAME                                          Контент-менеджер
ORGANIZATION                             Забиров Руслан Артурович
LOCATION                                          Санкт-Петербург
WORKTIME                            Полная занятость, полный день
EXPERIANCE                                           не требуется
SALARY                                     от 40 000 руб. на руки
REQUERMENTS                                       Adobe Photoshop
Name: 1, dtype: object

In [105]:
matched.iloc[2]

SUBFIELDCODE                                             38.04.05
SUBFIELDNAME                       Информационные системы бизнеса
DEGREE                                                    Магистр
SUBJECT                                                   WEB 2.0
SEMESTER                                                        2
START_YEAR                                                   2015
KEYWORDS        позиционирование, разметка, html, декоративные...
JOBNAME                         Web программист / веб-разработчик
ORGANIZATION                                            ProfitKit
LOCATION        Площадь Александра Невского 2, Санкт-Петербург...
WORKTIME                            Полная занятость, полный день
EXPERIANCE                                               1–3 года
SALARY                           от 30 000 до 50 000 руб. на руки
REQUERMENTS     HTML, CSS, PHP, jQuery, 1С-Битрикс, CMS Wordpr...
Name: 2, dtype: object

In [107]:
matched.iloc[5]

SUBFIELDCODE                                             01.04.02
SUBFIELDNAME    Экстренные вычисления и обработка сверхбольших...
DEGREE                                                    Магистр
SUBJECT               Методы и модели многомерного анализа данных
SEMESTER                                                        2
START_YEAR                                                   2015
KEYWORDS        Introduction to R; Multivariate Data Handling ...
JOBNAME                                          Программист Java
ORGANIZATION                                               СПЕКТР
LOCATION        Московская, Ленинский проспект, Санкт-Петербур...
WORKTIME                            Полная занятость, полный день
EXPERIANCE                                                3–6 лет
SALARY                          от 70 000 до 110 000 руб. на руки
REQUERMENTS     Java, Spring Framework, SQL, JMS, JAX-WS, JAX-...
Name: 5, dtype: object

# Performing simple matching

In [1]:
import numpy as np

def read_glove(filename):
    word_embeddings = {}
    f = open(filename,'r', errors = 'ignore', encoding='utf8')
    
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
        
    f.close()
    
    return word_embeddings

def read_eng_glove(filename):
    embedding_model = {}
    f = open(filename, encoding="utf8")
    for line in f:
        values = line.split()
        word = ''.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embedding_model[word] = coefs
    f.close()
    return embedding_model

In [2]:
# getting russian word embeddings
ru_word_embeddings = read_glove('multilingual_embeddings.ru')